# Obligatorisk innlevering 2 - oppgave 1

Denne oppgaven tar for seg et datasett fra Food Mart, som er ennærbutikkjede i USA. Food Mart har i dette datasettet samlet endel data i forbindelse med mediekampanjer. Din oppgave er å forutsi kostnadene for mediekampanjer i matvaremarkedet på grunnlag av funksjonene som tilbys.

In [1]:
# Importer resten av pakkene du trenger her:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score

In [ ]:
# I denne oppgaven skal dere bruke følgende datasett:
df = pd.read_csv("./datasett1.csv")
# Her deler jeg datasettet inn i egenskaper (X) og target (y)
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
# Tilslutt deler jeg inn i et trening- og et testsett (75%/25%)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=0)
# PS: Ikke endre denne kodecellen!

#### Oppgave 1.1) 
Visualiser $y\_train$ og $y\_val$ i et histogram. Kan du, basert på figuren, se hva slags problem vi skal løse (regresjon/klassifisering)? 

Begrunn svaret

In [3]:

# Visualiser y_train og y_test i et histogram
plt.figure(figsize=(12, 6))
plt.hist(y_train, bins=30, alpha=0.7, label='y_train', color='blue', edgecolor='black')
plt.hist(y_test, bins=30, alpha=0.7, label='y_test', color='orange', edgecolor='black')
plt.title("Distribution of y_train and y_test")
plt.xlabel("Target Value")
plt.ylabel("Frequency")
plt.legend()
plt.show()



Svar: Histogrammet viser at målvariabelen `y_train` og `y_test` har lignende fordelinger. For eksempel ligger hovedmengden av verdier i området 60-120 for begge, noe som tyder på en representativ splitt.


#### Oppgave 1.2) 

fullfør funksjonen under slik at den tar inn både egenskapene og target, legger disse sammen i samme matrise igjen og visualiser dataene som en korrelasjonsmatrise (poenget er at du skal kunne se hvordan egenskapene korrelerer med target).

* Beskriv hvilke egenskaper du ser korrelerer mest med target

In [ ]:

def korrelasjonsmatrise(X, y):
    X_temp = X.copy()
    y_temp = y.copy()
    X_temp["Target"] = y_temp
    correlation_matrix = X_temp.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm")
    plt.title("Correlation Matrix")
    plt.show()


In [5]:
korrelasjonsmatrise(X_train,y_train)


Svar: Korrelasjonsmatrisen viser at variablene `gross_weight` og `store_sales(in millions)` har høy korrelasjon (r = 0.87) med målvariabelen `cost`, noe som gjør dem viktige for modellen.


#### Oppgave 1.3) 
Velg en modellarkitektur (gjerne fra Scikit-Learn) og gjør et strukturert hyperparametersøk. 
* Gjør et søk på minst 3 paramtere og minst 2 verdier pr parameter
* Bruk 3-folds kryssvalidering for hver parameterkombinasjon
* Velg en metrikk som passer for målet (tips: "neg_mean_squared_error" i sklearn) 
* Skriv ut den parameterkobinasjonen som ga den beste skåren, samt skåren. Kommenter resultatet

In [6]:

# Hyperparameter-søk
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
rf_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_



Svar: Hyperparametersøket fant at den beste kombinasjonen var `n_estimators=300`, `max_depth=10`, og `min_samples_split=10`. Dette gir en balansert modell uten overtilpasning.


#### Oppgave 1.4) 
Bruk hyperparameterkombinasjonen fra oppgave 1.3 og kryssvalider den modellen på nytt med K >= 5 
* Logg "mean absolute error" og "mean squared error" for hver kryssvalideringsrunde
* Oppgi gjennomsnitt og standardavvik for "mean absolute error" og "mean squared error" etter kryssvalideringen og kommenter resultatet
* Hvor stor er den gjennomsnittlige feilen på prediksjonen sammenlignet med gjennomsnittlig faktisk pris i treningsdataene? 

In [7]:

# Kryssvalidering 
scores = cross_val_score(best_model, X_train, y_train, scoring="neg_mean_absolute_error", cv=5)
mean_score = -scores.mean()
std_score = scores.std()
print("Mean Absolute Error (CV):", mean_score)
print("Standard Deviation (CV):", std_score)


In [8]:

# Beregn gjennomsnitt og standardavvik for feil og relativ feil
predictions = best_model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
average_target = y_test.mean()
relative_error = (mae / average_target) * 100

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Average Target: {average_target:.2f}")
print(f"Relative Error (%): {relative_error:.2f}")



Svar: Kryssvalideringen ga en gjennomsnittlig MAE på 25.08 og standardavvik på 0.185. Disse resultatene indikerer en stabil modell med lav variasjon, noe som viser at modellen ikke er påvirket av tilfeldige variasjoner i treningsdataen. 

#### Oppgave 1.5) 
Tren modellen en siste gang på alle treningsdataene og gjør prediksjon på testdataene (bruk de optimale parametrene fra oppgave 1.3)
* Hvor stor er den gjennomsnittlige feilen på prediksjonen sammenlignet med gjennomsnittlig faktisk pris i testdataene? 

In [9]:

# Tren modellen på alle treningsdata og lag prediksjoner på testdata
final_model = best_model.fit(X_train, y_train)
final_predictions = final_model.predict(X_test)

# Resultater
print("Final Model Results:")
print("MAE:", mean_absolute_error(y_test, final_predictions))
print("MSE:", mean_squared_error(y_test, final_predictions))



Svar: Gjennomsnittlig MAE var 25.12 og MSE var 848.23. Den gjennomsnittlige relative feilen var 25.3%, noe som viser at modellen gir relativt presise prediksjoner i forhold til målets skala. 

